In [ ]:
from bs4 import BeautifulSoup
import requests
import json

In [ ]:
url = 'http://redistricting.lls.edu/who-partyfed.php'

# use html5lib to parse the broken html table
soup = BeautifulSoup(requests.get(url).content, 'html5lib')

In [ ]:
table =  soup.findAll('table')[1]

headers = []
for th in table.findAll('th'):
    #for cont in tr.findAll('td'):
        #if len(cont.text)>1:
    headers.append(th.text)
print headers

In [ ]:
# states_info = table.findAll('tr', {'class': 'odd'})

states_info = table.findAll('tr')


In [ ]:
#dictionary between full state names and abbreviations
with open("../data/state2St.json","rb") as f4:
    states = json.load(f4)

#dictionary between FIPS and states
with open("../data/st2Fips.json","rb") as f3:
    fips = json.load(f3)

states_redist2010 = {}
for info in states_info:
    state = info.findAll('td')
    if len(state) > 0:
        row = {}
        row['state'] = states[state[0].text]
        row['seats'] = int(state[1].text)

        ctrl = state[2].text
        if 'vetoproof' in ctrl:
            row['vetoproof'] = True
        else:
            row['vetoproof'] = False
        
        if 'Democratic' in ctrl:
            ctrl = 'D'
        elif 'Republican' in ctrl:
            ctrl = 'R'
        elif 'One congressional district' in ctrl and row['seats'] == 1:
            ctrl = 'OneDistrict'
        elif 'Split' in ctrl: # bipartisan commission, either independent or political
            ctrl = 'Split'
        elif 'Nonpartisan' in ctrl: # Nebraska only
            ctrl = 'Nonpartisan'
        else:
            #ctrl = 'ERROR'
            print 'Did not find redistricting control case for ' + row['state']
        row['redist_ctrl'] = ctrl
        
        row['governor_party'] = 'NA'
        row['redist_committee'] = 'NA'
        upper_D = 0
        upper_R = 0
        upper_O = 0
        lower_D = 0
        lower_R = 0
        lower_O = 0
        if len(state) == 4:
            row['redist_committee'] = state[3].text
        elif len(state) > 4:
            # all states except those with one district or a bipartisan commission
            if state[3].text == 'D':
                row['governor_party'] = 'D'
            elif state[3].text == 'R':
                row['governor_party'] = 'R'
            elif state[3].text == 'I': # Rhode Island
                row['governor_party'] = 'I'
            elif state[3].text == 'No veto': # Connecticuit and North Carolina have "no veto"
                row['governor_party'] = 'NoVeto'
            else:
                print 'No governor party found for ' + row['state']
            
            if len(state) == 5: # Nebraska
                row['redist_committee'] = state[4].text
            elif len(state) == 6:
                row['redist_committee'] = 'Legislative'
                upper = state[4].text.split(', ')
                for p in upper:
                    if p[-1] == 'D':
                        upper_D = int(p.split(' ')[0])
                    elif p[-1] == 'R':
                        upper_R = int(p.split(' ')[0])
                    elif p[-1] == 'O':
                        upper_O = int(p.split(' ')[0])

                lower = state[5].text.split(', ')
                for p in lower:
                    if p[-1] == 'D':
                        lower_D = int(p.split(' ')[0])
                    elif p[-1] == 'R':
                        lower_R = int(p.split(' ')[0])
                    elif p[-1] == 'O':
                        lower_O = int(p.split(' ')[0])
        
        row['upper_D'] = upper_D
        row['upper_R'] = upper_R
        row['upper_O'] = upper_O
        row['lower_D'] = lower_D
        row['lower_R'] = lower_R
        row['lower_O'] = lower_O

        #row = []
        #for item in state:
        #    row.append(item.text)
        states_redist2010[fips[row['state']]] = row

In [ ]:
with open('../data/congress/redistricting_2010.json', 'wb') as f:
    f.write(json.dumps(states_redist2010))